In [1]:
from pyspark.sql import SparkSession
spark = (
    SparkSession
    .builder
    .appName("WindowFunctions")
    .master("local[*]")
    .getOrCreate()
)

spark

In [19]:
# Emp Data & Schema

emp_data = [
    ["001","101","John Doe","30","Male","50000","2015-01-01"],
    ["002","101","Jane Smith","25","Female","45000","2016-02-15"],
    ["003","102","Bob Brown","35","Male","55000","2014-05-01"],
    ["004","102","Alice Lee","28","Female","48000","2017-09-30"],
    ["005","103","Jack Chan","40","Male","60000","2013-04-01"],
    ["006","103","Jill Wong","32","Female","52000","2018-07-01"],
    ["007","101","James Johnson","42","Male","10000","2012-03-15"],
    ["008","102","Kate Kim","29","Female","51000","2019-10-01"],
    ["009","103","Tom Tan","33","Male","58000","2016-06-01"],
    ["010","104","Lisa Lee","27","Female","47000","2018-08-01"],
    ["011","104","David Park","38","Male","65000","2015-11-01"],
    ["012","105","Susan Chen","31","Female","54000","2017-02-15"],
    ["013","106","Brian Kim","45","Male","75000","2011-07-01"],
    ["014","107","Emily Lee","26","Female","46000","2019-01-01"],
    ["015","106","Michael Lee","37","Male","63000","2014-09-30"],
    ["016","107","Kelly Zhang","30","Female","49000","2018-04-01"],
    ["017","105","George Wang","34","Male","57000","2016-03-15"],
    ["018","104","Nancy Liu","29","","50000","2017-06-01"],
    ["019","103","Steven Chen","36","Male","62000","2015-08-01"],
    ["020","102","Grace Kim","32","Female","53000","2018-11-01"]
]

emp_schema = "employee_id string, department_id string, name string, age string, gender string, salary string, hire_date string"

df = spark.createDataFrame(data = emp_data, schema = emp_schema)

df.printSchema()

df.show(5)

root
 |-- employee_id: string (nullable = true)
 |-- department_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: string (nullable = true)
 |-- hire_date: string (nullable = true)

+-----------+-------------+----------+---+------+------+----------+
|employee_id|department_id|      name|age|gender|salary| hire_date|
+-----------+-------------+----------+---+------+------+----------+
|        001|          101|  John Doe| 30|  Male| 50000|2015-01-01|
|        002|          101|Jane Smith| 25|Female| 45000|2016-02-15|
|        003|          102| Bob Brown| 35|  Male| 55000|2014-05-01|
|        004|          102| Alice Lee| 28|Female| 48000|2017-09-30|
|        005|          103| Jack Chan| 40|  Male| 60000|2013-04-01|
+-----------+-------------+----------+---+------+------+----------+
only showing top 5 rows



In [3]:
# select distinct * from <table>
df.distinct().show(5)

+-----------+-------------+----------+---+------+------+----------+
|employee_id|department_id|      name|age|gender|salary| hire_date|
+-----------+-------------+----------+---+------+------+----------+
|        001|          101|  John Doe| 30|  Male| 50000|2015-01-01|
|        003|          102| Bob Brown| 35|  Male| 55000|2014-05-01|
|        002|          101|Jane Smith| 25|Female| 45000|2016-02-15|
|        005|          103| Jack Chan| 40|  Male| 60000|2013-04-01|
|        004|          102| Alice Lee| 28|Female| 48000|2017-09-30|
+-----------+-------------+----------+---+------+------+----------+
only showing top 5 rows



In [4]:
#select distinct employee from table

df.select("department_id").distinct().show()

+-------------+
|department_id|
+-------------+
|          101|
|          102|
|          103|
|          104|
|          106|
|          105|
|          107|
+-------------+



In [25]:
# Window Functions
# select *, max(salary) over(partition by department_id order by salary desc) as max_salary from emp_unique

from pyspark.sql.window import Window
from pyspark.sql.functions import col, max

window_specification = Window.partitionBy(col("department_id")).orderBy(col("salary").desc())
window_function = max(col("salary")).over(window_specification)
df.withColumn("max_salary", window_function).show()




+-----------+-------------+-------------+---+------+------+----------+----------+
|employee_id|department_id|         name|age|gender|salary| hire_date|max_salary|
+-----------+-------------+-------------+---+------+------+----------+----------+
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|     50000|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|     50000|
|        007|          101|James Johnson| 42|  Male| 10000|2012-03-15|     50000|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|     55000|
|        020|          102|    Grace Kim| 32|Female| 53000|2018-11-01|     55000|
|        008|          102|     Kate Kim| 29|Female| 51000|2019-10-01|     55000|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|     55000|
|        019|          103|  Steven Chen| 36|  Male| 62000|2015-08-01|     62000|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|     62000|
|        009|   

In [26]:
# Window Functions - 2nd highest salary of each department
# select *, row_number() over (partition by department_id order by salary desc) as rn from <table> where rn = 2
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, col

window_spec = Window.partitionBy(col("department_id")).orderBy(col("salary").desc())
window_function = rank().over(window_spec)
df.withColumn("salary_rank_2", window_function).where("salary_rank_2 == 2").show()



+-----------+-------------+-----------+---+------+------+----------+-------------+
|employee_id|department_id|       name|age|gender|salary| hire_date|salary_rank_2|
+-----------+-------------+-----------+---+------+------+----------+-------------+
|        002|          101| Jane Smith| 25|Female| 45000|2016-02-15|            2|
|        020|          102|  Grace Kim| 32|Female| 53000|2018-11-01|            2|
|        005|          103|  Jack Chan| 40|  Male| 60000|2013-04-01|            2|
|        018|          104|  Nancy Liu| 29|      | 50000|2017-06-01|            2|
|        012|          105| Susan Chen| 31|Female| 54000|2017-02-15|            2|
|        015|          106|Michael Lee| 37|  Male| 63000|2014-09-30|            2|
|        014|          107|  Emily Lee| 26|Female| 46000|2019-01-01|            2|
+-----------+-------------+-----------+---+------+------+----------+-------------+



In [28]:
# Window function using expr
# select *, row_number() over(partition by department_id order by salary desc) as rn from emp_unique where rn = 2
from pyspark.sql.functions import expr

df.withColumn("rn", expr("row_number() over(partition by department_id order by salary desc)")).where("rn = 2").show()

+-----------+-------------+-----------+---+------+------+----------+---+
|employee_id|department_id|       name|age|gender|salary| hire_date| rn|
+-----------+-------------+-----------+---+------+------+----------+---+
|        002|          101| Jane Smith| 25|Female| 45000|2016-02-15|  2|
|        020|          102|  Grace Kim| 32|Female| 53000|2018-11-01|  2|
|        005|          103|  Jack Chan| 40|  Male| 60000|2013-04-01|  2|
|        018|          104|  Nancy Liu| 29|      | 50000|2017-06-01|  2|
|        012|          105| Susan Chen| 31|Female| 54000|2017-02-15|  2|
|        015|          106|Michael Lee| 37|  Male| 63000|2014-09-30|  2|
|        014|          107|  Emily Lee| 26|Female| 46000|2019-01-01|  2|
+-----------+-------------+-----------+---+------+------+----------+---+



In [29]:
spark.stop()